In [11]:
import os
import boto3

In [12]:
keys = [] 
iter_items = _list_bucket("canonical-rebuilt-versioned", prefix="GDL/1950", accept_key=lambda key: key.endswith('.json'))
for k in iter_items:
    keys.append(k)
print(len(keys)) 

dict_keys(['ResponseMetadata', 'IsTruncated', 'Contents', 'Name', 'Prefix', 'MaxKeys'])
1000


In [13]:
keys = [] 
iter_items= _list_bucket_paginator("canonical-rebuilt-versioned", prefix="GDL/1950", accept_key=lambda key: key.endswith('.json'))
print(len(iter_items)) 

4506


In [14]:
def _list_bucket(bucket_name, prefix='', accept_key=lambda k: True):
    client = get_s3_client()
    ctoken = None

    while True:
        response = client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
        try:
            content = response['Contents']
            print(response.keys())
            #print(response.get('NextContinuationToken', None))
        except KeyError:
            pass
        else:
            for c in content:
                key = c['Key']
                if accept_key(key):
                    yield key
        ctoken = response.get('NextContinuationToken', None)
        if not ctoken:
            break

In [15]:
def _list_bucket_paginator(bucket_name, prefix='', accept_key=lambda k: True):
    client = get_s3_client()
    paginator = client.get_paginator("list_objects")
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)
    keys = []
    for page in page_iterator:
        if "Contents" in page:
            for key in page["Contents"]:
                keyString = key["Key"]
                #yield key
                keys.append(keyString)
    return keys if keys else []

In [16]:
def get_s3_client(host_url='https://os.zhdk.cloud.switch.ch/'):
    if host_url is None:
        try:
            host_url = os.environ["SE_HOST_URL"]
        except Exception:
            raise

    try:
        access_key = os.environ["SE_ACCESS_KEY"]
    except Exception:
        raise

    try:
        secret_key = os.environ["SE_SECRET_KEY"]
    except Exception:
        raise

    return boto3.client(
        's3',
        aws_secret_access_key=secret_key,
        aws_access_key_id=access_key,
        endpoint_url=host_url
    )